In [1]:
from langchain_neo4j import Neo4jGraph
from langchain_core.prompts import ChatPromptTemplate

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile", api_key='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')

In [4]:
graph = Neo4jGraph(
    url="neo4j+s://00601af8.databases.neo4j.io", username="neo4j", password="oIBWHe2qEKESesEC1pBzwXL041YajSkKN5BIacTIIE0"
)

In [14]:
def extract_player_social_diversity_features(player_id: str) -> dict:
    """
    Extracts features from the knowledge graph for a given player.
    """
    query = f"""
    MATCH (p:Player {{Actor: toInteger('{player_id}')}})
    RETURN
        p.Actor AS player_id,
        p.A_Acc AS a_acc,
        p.Avg_PartyTime AS avg_partytime,
        p.GuildAct_count AS guild_act_count,
        p.GuildJoin_count AS guild_join_count
    """
    results = graph.query(query)
    if results:
        return results[0]
    else:
        return {}

In [35]:
prompt = ChatPromptTemplate.from_template(
"""
<|system|>
You are an expert game analyst tasked with identifying bots in an online game by analyzing player statistics. Your role is to generate an anomaly score (0-100, higher score means more likely to be a bot) and provide concise reasoning based on the player data.

Here is how to interpret each player statistic. Provide context for how each statistic informs your score. Focus on how these characteristics point to a bot.

- **Actor**: The player's unique identifier.
- **A_Acc**: Account ID; extreme values may indicate automated account creation.

- **Avg_PartyTime**: The average time spent in group activities.
  - Low: **0** (No party activity)
  - High: Above **5,358** (Excessively high group activity)
  - Bots tend to avoid parties or farm in groups for long hours.

- **GuildAct_count**: Number of guild activities.
  - Low: **0** (No guild activity)
  - High: **1+** (Some activity)
  - Bots often avoid guild activities unless scripted for farming.

- **GuildJoin_count**: Number of times a player has joined a guild.
  - Low: **0** (Never joined a guild)
  - High: **N/A** (Most values are 0)

Bot Indicators:
1. **Players with both extremely low or high Avg_PartyTime** tend to be bots.
2. **Zero guild activity (GuildAct_count = 0) strongly suggests bot-like behavior.**
3. **Accounts with very high or very low A_Acc values** may indicate automation.
4. **A mix of these factors increases the likelihood of bot activity.**


Based on these descriptions, you will be given the following information and need to determine these two things:

1. **Determine the anomaly score (0-100)**: A higher score indicates a greater likelihood of the player being a bot. This MUST be provided.  
2. **Provide a detailed explanation of your reasoning.** This MUST be provided.

Your final output **MUST** follow this structure:

Anomaly Score: [your anomaly score 0-100]  
Reasoning: [Your reasoning based on patterns from the statistics. Be as clear as possible]

Respond concisely and directly.

<|user|>
Here is the player data:

Actor: {actor}
A_Acc: {a_acc}  
Avg_PartyTime: {avg_partytime}  
GuildAct_count: {guildact_count}  
GuildJoin_count: {guildjoin_count}  

Anomaly Score:  
Reasoning:  

""")

In [36]:
def assess_bot_likelihood(player_data) -> tuple[int, str, str]:
    """
    Assesses the likelihood of a player being a bot using LLM and extracts score and reasoning.
    Returns a tuple of (anomaly_score, reasoning, full_analysis).
    """
    # Format the prompt
    formatted_prompt = prompt.format_messages(
        actor=player_data['player_id'],
        a_acc=player_data['a_acc'],
        avg_partytime=player_data['avg_partytime'],
        guildact_count=player_data['guild_act_count'],
        guildjoin_count=player_data['guild_join_count']
    )

    # Call the LLM directly
    response = llm.invoke(formatted_prompt)
    full_analysis = response.content

    # Extract Anomaly Score and Reasoning
    try:
        score_line = next(line for line in full_analysis.split('\n') if "Anomaly Score" in line)
        anomaly_score = int(score_line.split(":")[1].strip())

        reasoning = "\n".join(full_analysis.split('\n')[1:])  # All lines after the score
    except Exception as e:
        anomaly_score = None
        reasoning = f"Could not reliably parse LLM response: {str(e)}"

    return anomaly_score, reasoning, full_analysis

In [37]:
def generate_bot_report(player_ids: list[str]) -> list[dict]:
    """
    Generates a report for a list of player IDs, including anomaly scores and reasoning.
    """
    report = []
    for player_id in player_ids:
        player_data = extract_player_social_diversity_features(player_id)
        if player_data:
            anomaly_score, reasoning, full_analysis = assess_bot_likelihood(player_data)

            report.append({
                "player_id": player_id,
                "anomaly_score": anomaly_score,
                "reasoning": reasoning,
                #"full_analysis": /full_analysis  # Optional - adds full LLM output
            })
        else:
            report.append({
                "player_id": player_id,
                "anomaly_score": None,
                "reasoning": "No data found for this player.",
                "full_analysis": None
            })
    return report

In [38]:
player_ids_to_check = ['300978']  # Replace with actual player IDs
bot_report = generate_bot_report(player_ids_to_check)

for record in bot_report:
    print(f"Player ID: {record['player_id']}")
    print(f"Anomaly Score: {record['anomaly_score']}")
    print(f"Reasoning: {record['reasoning']}")
    print("-" * 40)

Player ID: 300978
Anomaly Score: 90
Reasoning: Reasoning: The player data suggests a high likelihood of bot-like behavior due to several key indicators. Firstly, the extremely high Avg_PartyTime of 5956.6364 is a strong bot indicator, as it suggests the player is farming in groups for excessively long hours, which is a common bot behavior. Additionally, the GuildAct_count of 0 implies that the player avoids guild activities, which is another characteristic often associated with bots. The A_Acc value of 7103528, while not extremely low, is on the higher side, which could indicate automated account creation. Lastly, the GuildJoin_count of 0 further supports the notion of bot-like behavior, as bots often do not engage in social aspects of the game like guild membership. The combination of these factors, particularly the extremely high Avg_PartyTime and complete lack of guild activity, leads to a high anomaly score, suggesting that this player is very likely a bot.
------------------------